In [7]:
import os
import glob
import numpy as np
import pandas as pd
import pickle
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

sns.set_context('poster')
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# import Python3_OpenOE_AC_map_functions_v1_08_30s as oem
import mz_LFP_functions as mz_LFP

import matplotlib.animation as ani #this was a good idea that I didn't want to spend time on right now

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Load some necessary variables

In [8]:
insert_depth = 1000  #change this as appropriate
sp_bw_ch = 20/2

samples_tr = 7350 #this is based on the shortest #samples in a trial
sr = 2500
n_chan = 384
rec_length = 3.0 #how long is the arduino triggered?

---

# For multiple mice
Requires a __`user input`__ to choose the scenario

In [31]:
data_choice = input('Scenario (pre, post, novel): ')

if data_choice == 'pre':
    start_path_ls=glob.glob(r"G:/Neuropixels/NMDA_V1HPC/"+'pre*')     # pre-training to the visual stim
    total_rec = 34
elif data_choice == 'post':
    start_path_ls=glob.glob(r"G:/Neuropixels/NMDA_V1HPC/"+'post*')     # post-training to the visual stim
    total_rec = 34
elif data_choice == 'novel':
    start_path_ls=glob.glob(r"G:/Neuropixels/NMDA_V1HPC/"+'novel*')     # novel stimulus
    total_rec = 34
else:
    raise Exception('Input is not one of the options (pre, post, novel)')


end_path = r"\continuous\Neuropix-PXI-100.1\continuous.dat"

Scenario (pre, post, novel): novel


In [32]:
alldatals=[]
et_ls = []
for start_path in start_path_ls:
    ls = []
    et = start_path.split('\\')[-1].split('_')[2]
    
    for i in range(6, total_rec): # remember, you have to exclude the first short TTL signal trials
        exp_rec_path = rf"\experiment1\recording{i}"
        fileName = start_path + exp_rec_path + end_path
        data = np.memmap(fileName, dtype='int16', mode='c')
        data2 = data.reshape(-1, n_chan)
        ls.append(data2[:samples_tr, 0:384])
        
    alldatals.append(ls)
    et_ls.append(et)

# alldatals: num_mice x num_trials x num_samples x num_ch
num_mice = len(et_ls)
print(num_mice)
et_ls


11


['et1710',
 'et171',
 'et1700',
 'et170',
 'et152',
 'et2750',
 'et275',
 'et630',
 'et63',
 'et2760',
 'et276']

---

# Transform the lists into CAR filtered arrays

In [33]:
times = np.linspace(0, samples_tr/sr, samples_tr)
scale_factor = 0.195
time_arr = [0,0.5,1,1.5,2,2.5]
time_plot = [i*sr for i in time_arr]

In [34]:
# uses "applyCAR" and "notch_filt" functions
def create_all_arr(alldatals):
    novel_ls=[]
    for i in range(len(alldatals)):
        tmp2 = alldatals[i]                               # getting the individual trials
        tmp3_ls = []
        for ii in range(len(tmp2)):
            tmp3 = tmp2[ii]
            tmp3 = tmp3.T
            filt_tmp3 = []
            for ch in range(tmp3.shape[0]):
                ch_data = tmp3[ch,:]
                ch_notc_data = mz_LFP.notch_filt(ch_data)
                filt_tmp3.append(ch_notc_data)
            filt_tmp3 = np.array(filt_tmp3)
            CARfilt_tmp3 = mz_LFP.applyCAR(filt_tmp3, pr=0)
            scaled_CARfilt_tmp3 = CARfilt_tmp3*scale_factor
            tmp3_ls.append(scaled_CARfilt_tmp3)
        novel_ls.append(tmp3_ls)
        print('Loaded {0}'.format(i))
    final_arr = np.array(novel_ls)
    return final_arr

In [35]:
# uses "applyCAR" and "notch_filt" functions
def create_novel_arr(alldatals):
    novel_ls=[]
    for i in range(len(alldatals)):
        tmp2 = np.mean(alldatals[i], axis = 0)           # getting the mean traces over all trials
        tmp2 = tmp2.T
        filt_tmp2 = []
        for ch in range(tmp2.shape[0]):
            ch_data = tmp2[ch,:]
            ch_notc_data = mz_LFP.notch_filt(ch_data)
            filt_tmp2.append(ch_notc_data)
        filt_tmp2 = np.array(filt_tmp2)
        CARfilt_tmp2 = mz_LFP.applyCAR(filt_tmp2, pr=0)
        scaled_CARfilt_tmp2 = CARfilt_tmp2*scale_factor
        novel_ls.append(scaled_CARfilt_tmp2)
        print('Loaded {0}'.format(i))
    final_arr = np.array(novel_ls)
    return final_arr

In [36]:
len(alldatals) # # mice x # trials x # samples x # channels

11

In [37]:
all_arr = create_all_arr(alldatals)

Loaded 0
Loaded 1
Loaded 2
Loaded 3
Loaded 4
Loaded 5
Loaded 6
Loaded 7
Loaded 8
Loaded 9
Loaded 10


In [38]:
all_arr.shape

(11, 28, 384, 7350)

---

# Saving the LFP arrays and CC_ls

In [39]:


arr_save_path = r"D:\mz_Data\saved_dfs\HPC_nmda\lfp_npy"



In [40]:
if data_choice == 'pre':
    fn1 = r"pre_all_trials"
    out1 = arr_save_path + "\\" + fn1
    np.save(out1, all_arr)         # saving pre array
    pkl_fn = r"pre_et_ls"
elif data_choice == 'post':
    fn1 = r"post_all_trials"
    out1 = arr_save_path + "\\" + fn1
    np.save(out1, all_arr)         # saving post array
    pkl_fn = r"post_et_ls"
elif data_choice == 'novel':
    fn1 = r"novel_all_trials"
    out1 = arr_save_path + "\\" + fn1
    np.save(out1, all_arr)         # saving novel array
    pkl_fn = r"novel_et_ls"

In [41]:
pkl_out = arr_save_path + "\\" + pkl_fn

open_file = open(pkl_out, "wb")
pickle.dump(et_ls, open_file)
open_file.close()

---

---

---